# 使用**pyjaspar**获取数据并构建打分矩阵
#### 利用pyjaspar可以读取数据库
#### 可以通过TF的jaspar id 或名字获取数据，但因为通过名字会返回多个结果，所以没有添加这个功能
#### 由于使用了pyjaspar，需要对以下内容进行引用：
**Khan, A. (2021) ‘pyjaspar: A serverless interface to Biopython to access different versions of JASPAR database’. Available at: https://github.com/asntech/pyjaspar (Accessed: 23 April 2023).**


In [1]:
from pyjaspar import jaspardb
import pandas as pd
from math import log2
jdb_obj = jaspardb(release='JASPAR2022')
motif = jdb_obj.fetch_motif_by_id('MA0032.2')


### 获取TF名称
#### fetch后得到的motif包含了大量关于该TF的信息，将其转换为字符串后，使用split切分，通过索引获得名称

In [2]:
print(str(motif))

TF name	FOXC1
Matrix ID	MA0032.2
Collection	CORE
TF class	['Fork head/winged helix factors']
TF family	['FOX']
Species	9606
Taxonomic group	vertebrates
Accession	['Q12948']
Data type used	HT-SELEX
Medline	17993506
Matrix:
        0      1      2      3      4      5      6      7      8      9     10
A: 6452.00 14324.00 7893.00 4858.00 2496.00 13117.00 18489.00 18489.00 122.00 18489.00 6702.00
C: 772.00 728.00 2008.00 336.00 1652.00 5372.00 108.00  56.00 8782.00  27.00 1558.00
G: 304.00 4165.00 2422.00 13631.00 390.00  21.00   0.00 141.00 165.00 433.00 1107.00
T: 12038.00 1386.00 10596.00  51.00 18489.00  45.00  57.00 112.00 9708.00 184.00 11787.00





In [3]:
str(motif).split()

['TF',
 'name',
 'FOXC1',
 'Matrix',
 'ID',
 'MA0032.2',
 'Collection',
 'CORE',
 'TF',
 'class',
 "['Fork",
 'head/winged',
 'helix',
 "factors']",
 'TF',
 'family',
 "['FOX']",
 'Species',
 '9606',
 'Taxonomic',
 'group',
 'vertebrates',
 'Accession',
 "['Q12948']",
 'Data',
 'type',
 'used',
 'HT-SELEX',
 'Medline',
 '17993506',
 'Matrix:',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 'A:',
 '6452.00',
 '14324.00',
 '7893.00',
 '4858.00',
 '2496.00',
 '13117.00',
 '18489.00',
 '18489.00',
 '122.00',
 '18489.00',
 '6702.00',
 'C:',
 '772.00',
 '728.00',
 '2008.00',
 '336.00',
 '1652.00',
 '5372.00',
 '108.00',
 '56.00',
 '8782.00',
 '27.00',
 '1558.00',
 'G:',
 '304.00',
 '4165.00',
 '2422.00',
 '13631.00',
 '390.00',
 '21.00',
 '0.00',
 '141.00',
 '165.00',
 '433.00',
 '1107.00',
 'T:',
 '12038.00',
 '1386.00',
 '10596.00',
 '51.00',
 '18489.00',
 '45.00',
 '57.00',
 '112.00',
 '9708.00',
 '184.00',
 '11787.00']

In [4]:
str(motif).split()[2]

'FOXC1'

### 构建打分矩阵
#### 通过构建PWM矩阵来进行打分
#### 以下展示如何从PFM进行一系列计算得到PWM
#### PFM: position frequency matrix
#### 对motif使用counts方法即可获得PFM矩阵

In [5]:
print(motif.counts)

        0      1      2      3      4      5      6      7      8      9     10
A: 6452.00 14324.00 7893.00 4858.00 2496.00 13117.00 18489.00 18489.00 122.00 18489.00 6702.00
C: 772.00 728.00 2008.00 336.00 1652.00 5372.00 108.00  56.00 8782.00  27.00 1558.00
G: 304.00 4165.00 2422.00 13631.00 390.00  21.00   0.00 141.00 165.00 433.00 1107.00
T: 12038.00 1386.00 10596.00  51.00 18489.00  45.00  57.00 112.00 9708.00 184.00 11787.00



#### 将PFM矩阵转为dataframe格式，方便进行计算

In [6]:
df = pd.DataFrame(motif.counts).T
df

,0,1,2,3,4,5,6,7,8,9,10
A,6452.0,14324.0,7893.0,4858.0,2496.0,13117.0,18489.0,18489.0,122.0,18489.0,6702.0
C,772.0,728.0,2008.0,336.0,1652.0,5372.0,108.0,56.0,8782.0,27.0,1558.0
G,304.0,4165.0,2422.0,13631.0,390.0,21.0,0.0,141.0,165.0,433.0,1107.0
T,12038.0,1386.0,10596.0,51.0,18489.0,45.0,57.0,112.0,9708.0,184.0,11787.0


### PPM: position probability matrix
#### 若motif长度为n，df的shape为4*n，df_sum为长度为n的向量，此处的除法利用了广播机制

In [7]:
df_sum = df.sum()
df = df / df_sum
df

,0,1,2,3,4,5,6,7,8,9,10
A,0.329756,0.695239,0.344387,0.257364,0.108394,0.706925,0.991155,0.983562,0.006497,0.966341,0.316820
C,0.039456,0.035335,0.087613,0.017800,0.071742,0.289518,0.005790,0.002979,0.467700,0.001411,0.073650
G,0.015537,0.202155,0.105677,0.722134,0.016937,0.001132,0.000000,0.007501,0.008787,0.022631,0.052331
T,0.615251,0.067272,0.462324,0.002702,0.802927,0.002425,0.003056,0.005958,0.517015,0.009617,0.557200


### PWM：position weight matrix
- PPM: 0.1 -> PWM: -1.32
- PPM: 0.05 -> PWM: -2.32
#### 将PPM矩阵中的每一个概率值除以0.25（背景频率，在核酸序列中是除以0.25，在蛋白质序列中是除以0.05）
#### 接着取对数，此处为每一个值加上一个极小的值，为了避免有0而无法取对数的情况
##### 接下来尝试对于此处进行的除以背景频率及取对数操作进行解释：
##### 如果是一段随机的序列，那么每个位置上每种碱基出现的平均概率为0.25（背景频率）
##### 将PPM中的概率除以这一平均概率得到一个相对权重，越是大于一则表示这一位点对于这一碱基的偏好更高，小于一则表示这一位点对这一碱基不偏好
##### 换一种说法，如果碱基作为一个功能位点和作为一个随机位点的概率相同，则分数为0
##### 如果它更有可能成为一个功能位点而不是随机位点，则分数大于0
##### 如果它更有可能成为一个随机位点而不是一个功能位点，则分数小于0。
##### 接下来的取对数操作有两个作用
##### 一是如果要用原本的概率衡量一段序列成为motif的可能性，则需要将概率相乘，取对数后则变为了相加
##### 二是取对数可以将更可能是随机位点的碱基变为负数，反之为正数，在打分时可以对“表现好”的位点加分，“表现差”的位点罚分（这一条是我自己感觉的）

In [8]:
df = df / 0.25
df = df.applymap(lambda x: log2(x + 1e-20))
df

,0,1,2,3,4,5,6,7,8,9,10
A,0.399469,1.475580,0.462102,0.041881,-1.205637,1.499630,1.987182,1.976088,-5.265942,1.950604,0.341733
C,-2.663604,-2.822772,-1.512713,-3.811948,-1.801041,0.211723,-5.432310,-6.390937,0.903655,-7.468888,-1.763163
G,-4.008134,-0.306466,-1.242273,1.530338,-3.883709,-7.787203,-66.438562,-5.058740,-4.830357,-3.465552,-2.256203
T,1.299247,-1.893855,0.886976,-6.531840,1.683341,-6.687667,-6.354307,-5.390937,1.048279,-4.700214,1.156266


### PWM矩阵的相关引用
**Xia, X. (2012) ‘Position weight matrix, gibbs sampler, and the associated significance tests in motif characterization and prediction’, Scientifica, 2012, p. 917540. Available at: https://doi.org/10.6064/2012/917540.**
**Staden, R. (1984) ‘Computer methods to locate signals in nucleic acid sequences.’, Nucleic Acids Research, 12(1 Pt 2), pp. 505–519.**


### 正则表达式生成
#### 每次读取PWM的一列，将分数大于-2的碱基选出，加入正则表达式

In [9]:
regular_motif = ''
for i in range(len(df.columns)):
    temp_df = pd.DataFrame(df.iloc[:, i])
    temp_df.columns = ['0']
    temp_df = temp_df[temp_df['0'] > -2]
    regular_motif += '[' + ''.join(temp_df.index) + ']'
regular_motif

'[AT][AGT][ACGT][AG][ACT][AC][A][A][CT][A][ACT]'

### 对序列进行打分

In [10]:
score = 0
seq = 'AAAAAAAATAA'
for i in range(len(seq)):
    base_score = df.loc[seq[i], i]
    score += base_score
print(score)

9.976912367954498


In [1]:
import tkinter as tk

root = tk.Tk()

text = tk.Text(root)
text.pack()

text.insert('end', 'Hello, world!\n')
text.insert('end', 'This is a test message.\n')

# 定义一个名为"big"的tag，将字体大小设置为20
text.tag_configure('big', font=('Arial', 20))

# 在第一行的末尾加入"big" tag
text.tag_add('big', '1.0', '1.end')

root.mainloop()
